## Methodology per Blair Russell

We may want to rethink the denominator of our equation for housing cost burden.

"Right now it’s all housing units with a cost burden computed. 

Alternatively, you could use low-income households (with cost burden computed) as the denominator, which would be a measure of relative cost burden just for low-income households. 

Both approaches are appropriate, but they tell a different story. You can imagine an area with few low-income households but a vast majority of them being cost burdened. In your calculation, you’d get a small percentage. 

In the alternative approach, it’s a large percentage. Just something to think about. It depends on the story you want to tell."


#### Packages

In [2]:
import math
import numpy as np
import os
import pandas as pd

# Table of Contents

Comparisons
*  [Implementation by Lucas](#lucas)
*  [Implementation by Saran](#saran)
*  [Side-by-side Comparison](#comparison)

The relevant denominator variables - all with line type of subtotal - in table 8 of the CHAS dataset are the following (CHAS data dictionary available [here](https://www.huduser.gov/portal/datasets/cp/CHAS-data-dictionary-14-18.xlsx)):

|   Name  |                          Label                      |
|---------|-----------------------------------------------------|
|T1_est3  |      Owner occupied less or equal to 30% of HAMFI   |                                                    | 
|T8_est16 |            Owner occupied greater than 30% of HAMFI cost burden less than 50%           |
|T8_est29 |Owner occupied greater than 50% but less than or equal to 80% of HAMFI	cost burden |
|T8_est69 |Renter occupied less than or equal to 30% of HAMFI|
|T8_est82 |           Renter occupied less than or equal to 30% of HAMFI cost burden greater than 50%           |
|T8_est95 |Renter occupied	greater than 50% but less than or equal to 80% of HAMFI|
 

### Implementation by Lucas <a id='lucas'></a>

In [3]:
# Read in the data from https://www.huduser.gov/portal/datasets/cp.html
housing = pd.read_csv("Table8.csv", 
                      encoding = "ISO-8859-1",  
                      dtype = {'Tract_ID': object, 'st': object, 'geoid': object})


# Remove data for states that aren't included in the census (e.g. American Samoa, Guam, etc.):
housing.drop(housing.loc[housing['st'] == '72'].index, inplace = True)

# Rename columns
housing = housing.rename(columns = {'geoid' :'FIPS_tract_id',
                                    'st' : 'state'
                                   })

# Owner occupied numerator fields
OWNER_OCCUPIED_NUMERATOR_FIELDS = [
    # Column Name
    #   Line_Type
    #   Tenure
    #   Household income
    #   Cost burden
    #   Facilities
    "T8_est7",
    #   Subtotal
    #   Owner occupied
    #   less than or equal to 30% of HAMFI
    #   greater than 30% but less than or equal to 50%
    #   All
    "T8_est10",
    #   Subtotal
    #   Owner occupied
    #   less than or equal to 30% of HAMFI
    #   greater than 50%
    #   All
    "T8_est20",
    #   Subtotal
    #   Owner occupied
    #   greater than 30% but less than or equal to 50% of HAMFI
    #   greater than 30% but less than or equal to 50%
    #   All
    "T8_est23",
    #   Subtotal
    #   Owner occupied
    #   greater than 30% but less than or equal to 50% of HAMFI
    #   greater than 50%
    #   All
    "T8_est33",
    #   Subtotal
    #   Owner occupied
    #   greater than 50% but less than or equal to 80% of HAMFI
    #   greater than 30% but less than or equal to 50%
    #   All
    "T8_est36",
    #   Subtotal
    #   Owner occupied
    #   greater than 50% but less than or equal to 80% of HAMFI
    #   greater than 50%
    #   All
]

# These rows have the values where HAMFI was not computed, b/c of no or negative income.
OWNER_OCCUPIED_NOT_COMPUTED_FIELDS = [
    # Column Name
    #   Line_Type
    #   Tenure
    #   Household income
    #   Cost burden
    #   Facilities
    "T8_est13",
    #   Subtotal
    #   Owner occupied
    #   less than or equal to 30% of HAMFI
    #   not computed (no/negative income)
    #   All
    "T8_est26",
    #   Subtotal
    #   Owner occupied
    #   greater than 30% but less than or equal to 50% of HAMFI
    #   not computed (no/negative income)
    #   All
    "T8_est39",
    #   Subtotal
    #   Owner occupied
    #   greater than 50% but less than or equal to 80% of HAMFI
    #   not computed (no/negative income)
    #   All
    "T8_est52",
    #   Subtotal
    #   Owner occupied
    #   greater than 80% but less than or equal to 100% of HAMFI
    #   not computed (no/negative income)
    #   All
    "T8_est65",
    #   Subtotal
    #   Owner occupied
    #   greater than 100% of HAMFI
    #   not computed (no/negative income)
    #   All
]

OWNER_OCCUPIED_POPULATION_FIELD = "T8_est2"
#   Subtotal
#   Owner occupied
#   All
#   All
#   All

OWNER_OCCUPIED_POPULATION_HAMFI_FIELD = "T8_est3"
#   Subtotal
#   Owner occupied 
#   All
#   All
#   All

# Renter occupied numerator fields
RENTER_OCCUPIED_NUMERATOR_FIELDS = [
    # Column Name
    #   Line_Type
    #   Tenure
    #   Household income
    #   Cost burden
    #   Facilities
    "T8_est73",
    #   Subtotal
    #   Renter occupied
    #   less than or equal to 30% of HAMFI
    #   greater than 30% but less than or equal to 50%
    #   All
    "T8_est76",
    #   Subtotal
    #   Renter occupied
    #   less than or equal to 30% of HAMFI
    #   greater than 50%
    #   All
    "T8_est86",
    #   Subtotal
    #   Renter occupied
    #   greater than 30% but less than or equal to 50% of HAMFI
    #   greater than 30% but less than or equal to 50%
    #   All
    "T8_est89",
    #   Subtotal
    #   Renter occupied
    #   greater than 30% but less than or equal to 50% of HAMFI
    #   greater than 50%
    #   All
    "T8_est99",
    #   Subtotal
    #   Renter occupied	greater than 50% but less than or equal to 80% of HAMFI
    #   greater than 30% but less than or equal to 50%
    #   All
    "T8_est102",
    #   Subtotal
    #   Renter occupied
    #   greater than 50% but less than or equal to 80% of HAMFI
    #   greater than 50%
    #   All
]

# These rows have the values where HAMFI was not computed, b/c of no or negative income.
RENTER_OCCUPIED_NOT_COMPUTED_FIELDS = [
    # Column Name
    #   Line_Type
    #   Tenure
    #   Household income
    #   Cost burden
    #   Facilities
    "T8_est79",
    #   Subtotal
    #   Renter occupied	less than or equal to 30% of HAMFI
    #   not computed (no/negative income)
    #   All
    "T8_est92",
    #   Subtotal
    #   Renter occupied	greater than 30% but less than or equal to 50% of HAMFI
    #   not computed (no/negative income)
    #   All
    "T8_est105",
    #   Subtotal
    #   Renter occupied
    #   greater than 50% but less than or equal to 80% of HAMFI
    #   not computed (no/negative income)
    #   All
    "T8_est118",
    #   Subtotal
    #   Renter occupied	greater than 80% but less than or equal to 100% of HAMFI
    #   not computed (no/negative income)
    #   All
    "T8_est131",
    #   Subtotal
    #   Renter occupied
    #   greater than 100% of HAMFI
    #   not computed (no/negative income)
    #   All
]

# T8_est68	SubtotalRenter occupied	All	All	All
RENTER_OCCUPIED_POPULATION_FIELD = "T8_est68"

In [4]:
# Original computation:
# (
#     # of Owner Occupied Units Meeting Criteria
#     + # of Renter Occupied Units Meeting Criteria
# )
# divided by
# (
#     Total # of Owner Occupied Units
#     + Total # of Renter Occupied Units
#     - # of Owner Occupied Units with HAMFI Not Computed
#     - # of Renter Occupied Units with HAMFI Not Computed
# )

housing["numerator_pre"] = housing[
    OWNER_OCCUPIED_NUMERATOR_FIELDS
].sum(axis=1) + housing[RENTER_OCCUPIED_NUMERATOR_FIELDS].sum(axis=1)



housing["denominator_pre"] = (
    housing[OWNER_OCCUPIED_POPULATION_FIELD]
    + housing[RENTER_OCCUPIED_POPULATION_FIELD]
    - housing[OWNER_OCCUPIED_NOT_COMPUTED_FIELDS].sum(axis=1)
    - housing[RENTER_OCCUPIED_NOT_COMPUTED_FIELDS].sum(axis=1)
)

### Implementation by Saran <a id='saran'></a>

In [5]:
OWNER_REVISED_DENOMINATOR_FIELDS =   [
            # Column Name
            #   Line_Type
            #   Tenure
            #   Household income
            #   Cost burden
            #   Facilities
            "T8_est3",
            #   Subtotal
            #   Renter occupied	less than or equal to 30% of HAMFI
            #   All
            "T8_est16",
            #   Subtotal
            #   Renter occupied	greater than 30% but less than or equal to 50% of HAMFI
            #   All
            "T8_est29",
            #   Subtotal
            #   Renter occupied
            #   greater than 50% but less than or equal to 80% of HAMFI
            #   All
        ]

RENTAL_REVISED_DENOMINATOR_FIELDS =   [
            # Column Name
            #   Line_Type
            #   Tenure
            #   Household income
            #   Cost burden
            #   Facilities
            "T8_est69",
            #   Subtotal
            #   Renter occupied	less than or equal to 30% of HAMFI
            #   All
            "T8_est82",
            #   Subtotal
            #   Renter occupied	greater than 30% but less than or equal to 50% of HAMFI
            #   All
            "T8_est95",
            #   Subtotal
            #   Renter occupied
            #   greater than 50% but less than or equal to 80% of HAMFI
            #   All
        ]

# New computation:
# (
#     # of Owner Occupied Units Meeting Criteria
#     + # of Renter Occupied Units Meeting Criteria
# )
# divided by
# (
#     Total # of Owner Occupied Units that meet criterion for poverty
#     + Total # of Renter Occupied Units that meet criterion for poverty
#     - # of Owner Occupied Units with HAMFI Not Computed
#     - # of Renter Occupied Units with HAMFI Not Computed
# )

housing['denominator_post'] = housing[
    RENTAL_REVISED_DENOMINATOR_FIELDS
].sum(axis = 1) + housing[OWNER_REVISED_DENOMINATOR_FIELDS].sum(axis = 1) - (
    - housing[OWNER_OCCUPIED_NOT_COMPUTED_FIELDS].sum(axis=1)
    - housing[RENTER_OCCUPIED_NOT_COMPUTED_FIELDS].sum(axis=1)
)

In [6]:
housing.iloc[:, -3:].head()

,numerator_pre,denominator_pre,denominator_post
0,174,765,295
1,177,720,350
2,279,1291,559
3,274,1635,525
4,885,4135,1090


#### Concatenate GeoIDs with Derived Columns

In [7]:
housing_df = pd.concat([housing.iloc[:, 2: 7], housing.iloc[:, -3:]], axis = 1)

In [11]:
housing_df["ratio_pre"] = np.round(
    housing_df['numerator_pre'] / housing_df['denominator_pre'], 2)

In [12]:
housing_df["ratio_post"] = np.round(
    housing_df['numerator_pre'] / housing_df['denominator_post'], 2)

### Comparison <a id='comparison'></a>

In [13]:
housing_df.head()

,FIPS_tract_id,name,state,cnty,tract,numerator_pre,denominator_pre,denominator_post,ratio_pre,ratio_post
0,14000US01001020100,"Census Tract 201, Autauga County, Alabama",01,1,20100,174,765,295,0.23,0.59
1,14000US01001020200,"Census Tract 202, Autauga County, Alabama",01,1,20200,177,720,350,0.25,0.51
2,14000US01001020300,"Census Tract 203, Autauga County, Alabama",01,1,20300,279,1291,559,0.22,0.50
3,14000US01001020400,"Census Tract 204, Autauga County, Alabama",01,1,20400,274,1635,525,0.17,0.52
4,14000US01001020500,"Census Tract 205, Autauga County, Alabama",01,1,20500,885,4135,1090,0.21,0.81
